# QuantumFold-Advantage: Complete Benchmarking Suite

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Tommaso-R-Marena/QuantumFold-Advantage/blob/main/examples/complete_benchmark.ipynb)

**This notebook runs EVERYTHING:**
- Full training pipeline (quantum vs classical)
- Comprehensive evaluation metrics
- Statistical validation with hypothesis testing
- Publication-ready visualizations
- Ablation studies

**Estimated runtime:** 30-60 minutes with GPU

**Author:** Tommaso R. Marena (The Catholic University of America)

## 1. Setup Environment

In [ ]:
# Check GPU availability
import torch
print(f'PyTorch version: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
else:
    print('WARNING: No GPU detected. Training will be slow.')

# Mount Google Drive (optional, for saving results)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    SAVE_TO_DRIVE = True
    print('Google Drive mounted successfully!')
except:
    SAVE_TO_DRIVE = False
    print('Running without Google Drive')

In [ ]:
# Clone repository
!git clone https://github.com/Tommaso-R-Marena/QuantumFold-Advantage.git
import os
os.chdir('QuantumFold-Advantage')
print('Repository cloned successfully!')
!pwd

In [ ]:
# Install core dependencies
print('Installing core dependencies...')
!pip install -q torch pennylane pennylane-lightning
!pip install -q numpy scipy pandas matplotlib seaborn plotly
!pip install -q scikit-learn statsmodels biopython
!pip install -q tqdm pyyaml

# Install ESM-2 (optional but recommended)
print('\nInstalling ESM-2...')
!pip install -q fair-esm transformers

print('\nAll dependencies installed!')

## 2. Configuration

In [ ]:
# Experiment configuration
import sys
sys.path.insert(0, '/content/QuantumFold-Advantage')

CONFIG = {
    # Data
    'train_samples': 200,  # Increase for better results
    'val_samples': 50,
    'test_samples': 50,
    'seq_len': 64,
    
    # Training
    'epochs': 20,  # Increase to 50-100 for publication
    'batch_size': 16,
    'learning_rate': 1e-3,
    'warmup_epochs': 2,
    
    # Model
    'hidden_dim': 256,
    'pair_dim': 64,
    'n_structure_layers': 4,
    
    # Quantum
    'n_qubits': 6,
    'quantum_depth': 3,
    
    # Validation
    'alpha': 0.05,
    'n_bootstrap': 5000,
    
    # Output
    'output_dir': '/content/outputs',
    'save_to_drive': SAVE_TO_DRIVE
}

print('Configuration:')
for key, value in CONFIG.items():
    print(f'  {key}: {value}')

## 3. Data Preparation

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class ProteinDataset(Dataset):
    def __init__(self, n_samples, seq_len, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        self.n_samples = n_samples
        self.seq_len = seq_len
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        
        # Generate synthetic protein structures
        self.sequences = []
        self.coordinates = []
        
        for i in range(n_samples):
            # Random sequence
            seq = ''.join(np.random.choice(list(self.amino_acids), size=seq_len))
            self.sequences.append(seq)
            
            # Alpha helix coordinates with noise
            t = np.linspace(0, 4*np.pi, seq_len)
            coords = np.zeros((seq_len, 3))
            coords[:, 0] = 2.3 * np.cos(t) + np.random.randn(seq_len) * 0.3
            coords[:, 1] = 2.3 * np.sin(t) + np.random.randn(seq_len) * 0.3
            coords[:, 2] = 1.5 * t + np.random.randn(seq_len) * 0.3
            self.coordinates.append(coords)
    
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        return {
            'sequence': self.sequences[idx],
            'coordinates': torch.tensor(self.coordinates[idx], dtype=torch.float32)
        }

# Create datasets
print('Creating datasets...')
train_dataset = ProteinDataset(CONFIG['train_samples'], CONFIG['seq_len'], seed=42)
val_dataset = ProteinDataset(CONFIG['val_samples'], CONFIG['seq_len'], seed=123)
test_dataset = ProteinDataset(CONFIG['test_samples'], CONFIG['seq_len'], seed=456)

print(f'Train: {len(train_dataset)} samples')
print(f'Val: {len(val_dataset)} samples')
print(f'Test: {len(test_dataset)} samples')
print(f'Sequence length: {CONFIG["seq_len"]} residues')

## 4. Model Training

### 4.1 Train Quantum-Enhanced Model

In [ ]:
from src.advanced_model import AdvancedProteinFoldingModel
from src.advanced_training import AdvancedTrainer, TrainingConfig
from src.protein_embeddings import ESM2Embedder
from functools import partial
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('QuantumFold')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Load ESM-2 embedder (smaller model for Colab)
print('\nLoading ESM-2 embedder...')
embedder = ESM2Embedder(model_name='esm2_t12_35M_UR50D', freeze=True).to(device)
print(f'Embedding dimension: {embedder.embed_dim}')

# Collate function
def collate_fn(batch, embedder):
    sequences = [item['sequence'] for item in batch]
    coordinates = torch.stack([item['coordinates'] for item in batch])
    
    with torch.no_grad():
        embeddings_dict = embedder(sequences)
        embeddings = embeddings_dict['embeddings']
    
    return {
        'sequence': embeddings,
        'coordinates': coordinates,
        'mask': torch.ones(len(batch), embeddings.shape[1], dtype=torch.bool)
    }

collate_with_emb = partial(collate_fn, embedder=embedder)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], 
                         shuffle=True, collate_fn=collate_with_emb)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], 
                       shuffle=False, collate_fn=collate_with_emb)

print('\nDataloaders created successfully!')

In [ ]:
# Initialize quantum model
print('\n' + '='*80)
print('TRAINING QUANTUM-ENHANCED MODEL')
print('='*80)

quantum_model = AdvancedProteinFoldingModel(
    input_dim=embedder.embed_dim,
    c_s=CONFIG['hidden_dim'],
    c_z=CONFIG['pair_dim'],
    n_structure_layers=CONFIG['n_structure_layers'],
    use_quantum=True
)

n_params = sum(p.numel() for p in quantum_model.parameters() if p.requires_grad)
print(f'Model parameters: {n_params:,}')

# Training configuration
quantum_config = TrainingConfig(
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    learning_rate=CONFIG['learning_rate'],
    warmup_epochs=CONFIG['warmup_epochs'],
    use_amp=True,
    use_ema=True,
    checkpoint_dir=f"{CONFIG['output_dir']}/quantum_checkpoints"
)

# Train
quantum_trainer = AdvancedTrainer(quantum_model, quantum_config, device, logger)
quantum_history = quantum_trainer.train(train_loader, val_loader)

print('\nQuantum model training complete!')

### 4.2 Train Classical Baseline

In [ ]:
# Initialize classical model
print('\n' + '='*80)
print('TRAINING CLASSICAL BASELINE MODEL')
print('='*80)

classical_model = AdvancedProteinFoldingModel(
    input_dim=embedder.embed_dim,
    c_s=CONFIG['hidden_dim'],
    c_z=CONFIG['pair_dim'],
    n_structure_layers=CONFIG['n_structure_layers'],
    use_quantum=False  # Classical baseline
)

n_params_classical = sum(p.numel() for p in classical_model.parameters() if p.requires_grad)
print(f'Model parameters: {n_params_classical:,}')
print(f'Parameter reduction: {(1 - n_params/n_params_classical)*100:.1f}%')

# Training configuration
classical_config = TrainingConfig(
    epochs=CONFIG['epochs'],
    batch_size=CONFIG['batch_size'],
    learning_rate=CONFIG['learning_rate'],
    warmup_epochs=CONFIG['warmup_epochs'],
    use_amp=True,
    use_ema=True,
    checkpoint_dir=f"{CONFIG['output_dir']}/classical_checkpoints"
)

# Train
classical_trainer = AdvancedTrainer(classical_model, classical_config, device, logger)
classical_history = classical_trainer.train(train_loader, val_loader)

print('\nClassical model training complete!')

## 5. Comprehensive Evaluation

In [ ]:
from src.benchmarks import ProteinStructureEvaluator
from tqdm import tqdm

evaluator = ProteinStructureEvaluator()
test_loader = DataLoader(test_dataset, batch_size=CONFIG['batch_size'], 
                        shuffle=False, collate_fn=collate_with_emb)

def evaluate_model(model, loader, name):
    model.eval()
    
    all_tm_scores = []
    all_rmsd = []
    all_gdt_ts = []
    all_plddt = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc=f'Evaluating {name}'):
            inputs = batch['sequence'].to(device)
            coords_true = batch['coordinates'].to(device)
            
            # Predict
            outputs = model(inputs)
            coords_pred = outputs['coordinates']
            plddt = outputs['plddt']
            
            # Move to CPU
            coords_pred_np = coords_pred.cpu().numpy()
            coords_true_np = coords_true.cpu().numpy()
            plddt_np = plddt.cpu().numpy()
            
            # Calculate metrics
            for i in range(len(coords_pred_np)):
                tm = evaluator.calculate_tm_score(coords_pred_np[i], coords_true_np[i])
                rmsd = evaluator.calculate_rmsd(coords_pred_np[i], coords_true_np[i])
                gdt = evaluator.calculate_gdt_ts(coords_pred_np[i], coords_true_np[i])
                
                all_tm_scores.append(tm)
                all_rmsd.append(rmsd)
                all_gdt_ts.append(gdt)
                all_plddt.append(plddt_np[i].mean())
    
    return {
        'tm_score': np.array(all_tm_scores),
        'rmsd': np.array(all_rmsd),
        'gdt_ts': np.array(all_gdt_ts),
        'plddt': np.array(all_plddt)
    }

print('\n' + '='*80)
print('EVALUATION ON TEST SET')
print('='*80)

quantum_results = evaluate_model(quantum_model, test_loader, 'Quantum')
classical_results = evaluate_model(classical_model, test_loader, 'Classical')

print('\nResults Summary:')
print('\nQuantum Model:')
for metric, values in quantum_results.items():
    print(f'  {metric}: {values.mean():.4f} ± {values.std():.4f}')

print('\nClassical Model:')
for metric, values in classical_results.items():
    print(f'  {metric}: {values.mean():.4f} ± {values.std():.4f}')

## 6. Statistical Validation

In [ ]:
from src.statistical_validation import ComprehensiveBenchmark

print('\n' + '='*80)
print('STATISTICAL VALIDATION')
print('='*80)

benchmark = ComprehensiveBenchmark(
    output_dir=f"{CONFIG['output_dir']}/validation",
    alpha=CONFIG['alpha']
)

# TM-score comparison
print('\n### TM-Score Analysis ###')
tm_comparison = benchmark.compare_methods(
    quantum_scores=quantum_results['tm_score'],
    classical_scores=classical_results['tm_score'],
    metric_name='TM-score',
    higher_is_better=True
)

# RMSD comparison
print('\n### RMSD Analysis ###')
rmsd_comparison = benchmark.compare_methods(
    quantum_scores=quantum_results['rmsd'],
    classical_scores=classical_results['rmsd'],
    metric_name='RMSD',
    higher_is_better=False
)

# Generate visualizations
benchmark.plot_comparison(
    quantum_results['tm_score'],
    classical_results['tm_score'],
    metric_name='TM-score'
)

benchmark.plot_comparison(
    quantum_results['rmsd'],
    classical_results['rmsd'],
    metric_name='RMSD'
)

# Save results
benchmark.save_results()
benchmark.generate_report()

print('\nStatistical validation complete!')

## 7. Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Training curves
ax = axes[0, 0]
ax.plot(quantum_history['train_loss'], label='Quantum Train', linewidth=2)
ax.plot(quantum_history['val_loss'], label='Quantum Val', linewidth=2, linestyle='--')
ax.plot(classical_history['train_loss'], label='Classical Train', linewidth=2)
ax.plot(classical_history['val_loss'], label='Classical Val', linewidth=2, linestyle='--')
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Loss', fontsize=12)
ax.set_title('Training Curves', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# 2. TM-score distribution
ax = axes[0, 1]
ax.hist(quantum_results['tm_score'], bins=20, alpha=0.6, label='Quantum', color='blue')
ax.hist(classical_results['tm_score'], bins=20, alpha=0.6, label='Classical', color='orange')
ax.set_xlabel('TM-score', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('TM-score Distribution', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# 3. RMSD distribution
ax = axes[1, 0]
ax.hist(quantum_results['rmsd'], bins=20, alpha=0.6, label='Quantum', color='blue')
ax.hist(classical_results['rmsd'], bins=20, alpha=0.6, label='Classical', color='orange')
ax.set_xlabel('RMSD (Å)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('RMSD Distribution', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# 4. Scatter plot
ax = axes[1, 1]
ax.scatter(classical_results['tm_score'], quantum_results['tm_score'], alpha=0.6, s=100)
lim = [0, 1]
ax.plot(lim, lim, 'r--', linewidth=2, label='Equal performance')
ax.set_xlabel('Classical TM-score', fontsize=12)
ax.set_ylabel('Quantum TM-score', fontsize=12)
ax.set_title('Paired Comparison', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(f"{CONFIG['output_dir']}/summary_plots.png", dpi=300, bbox_inches='tight')
plt.show()

print('\nVisualization complete!')

## 8. Save Results

In [ ]:
import json
from pathlib import Path

# Save all results
results_summary = {
    'config': CONFIG,
    'quantum_metrics': {k: v.tolist() for k, v in quantum_results.items()},
    'classical_metrics': {k: v.tolist() for k, v in classical_results.items()},
    'statistical_tests': {
        'tm_score': tm_comparison,
        'rmsd': rmsd_comparison
    }
}

output_path = Path(CONFIG['output_dir']) / 'complete_results.json'
with open(output_path, 'w') as f:
    json.dump(results_summary, f, indent=2, default=str)

print(f'Results saved to {output_path}')

# Copy to Google Drive if mounted
if CONFIG['save_to_drive']:
    !mkdir -p /content/drive/MyDrive/QuantumFold_Results
    !cp -r {CONFIG['output_dir']}/* /content/drive/MyDrive/QuantumFold_Results/
    print('Results copied to Google Drive!')

# Create downloadable archive
!tar -czf results.tar.gz -C {CONFIG['output_dir']} .
print('\nCreated results.tar.gz for download')

from google.colab import files
try:
    files.download('results.tar.gz')
except:
    print('Download manually from files panel')

## 9. Final Summary

In [ ]:
print('='*80)
print('COMPLETE BENCHMARK SUMMARY')
print('='*80)

print(f'\nDataset: {CONFIG["train_samples"]} train, {CONFIG["val_samples"]} val, {CONFIG["test_samples"]} test')
print(f'Training epochs: {CONFIG["epochs"]}')
print(f'Model parameters: {n_params:,} (quantum), {n_params_classical:,} (classical)')

print('\nPerformance Metrics:')
print('\nQuantum Model:')
for metric in ['tm_score', 'rmsd', 'gdt_ts', 'plddt']:
    mean = quantum_results[metric].mean()
    std = quantum_results[metric].std()
    print(f'  {metric.upper()}: {mean:.4f} ± {std:.4f}')

print('\nClassical Model:')
for metric in ['tm_score', 'rmsd', 'gdt_ts', 'plddt']:
    mean = classical_results[metric].mean()
    std = classical_results[metric].std()
    print(f'  {metric.upper()}: {mean:.4f} ± {std:.4f}')

print('\nStatistical Significance:')
print(f'  TM-score p-value: {tm_comparison["wilcoxon"]["p_value"]:.4e}')
print(f'  TM-score Cohen\'s d: {tm_comparison["ttest"]["effect_size"]:.4f}')
print(f'  RMSD p-value: {rmsd_comparison["wilcoxon"]["p_value"]:.4e}')

if tm_comparison['wilcoxon']['p_value'] < CONFIG['alpha']:
    print('  ✅ Statistically significant quantum advantage detected!')
else:
    print('  ⚠️  No significant difference detected. Consider more training.')

print('\n' + '='*80)
print('BENCHMARK COMPLETE!')
print('='*80)